# IBM Data Science Professional Certificate

##### Applied Data Science Week 3 Assignment: Segmenting and Clustering Neighborhoods in Toronto
##### Coursera: https://www.coursera.org/learn/applied-data-science-capstone

##### Preprocessing: Prepare for arranged dataframe from section 1 & 2

In [259]:
from bs4 import BeautifulSoup
import requests
import lxml
import csv
import pandas as pd
import numpy as np
import geopy

In [255]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#table = soup.find('table', class_ ='wikitable sortable')
header = table.find_all('th')
hd = []
for h in header:
    head = h.text
    hd.append(head)
hd2 = [x.strip('\n') for x in hd]
hd2[0] = 'PostalCode'

rows = table.find_all('tr')
rw = []
for row in rows:
    cell = row.find_all('td')
    row = [i.text for i in cell]
    row2 = [x.strip('\n') for x in row]
    rw.append(row2)
rw2 = [x for x in rw if x != []]

df = pd.DataFrame(rw2)
df.columns = [hd2]

df.to_csv('df.csv', index=False)
df1 = pd.read_csv('df.csv')
df2 = df1.dropna(axis=0)
df2.reset_index(drop=True)
df3 = df2
df3.Neighborhood = df3.Neighborhood.str.replace('/',',') 

geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo = pd.DataFrame(geo_data)
geo = geo.rename(columns={'Postal Code':'PostalCode'})

df4 = df3.merge(geo,how='outer')
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [256]:
df4.shape

(103, 5)

### Section 3: Explore and cluster the neighborhoods in Toronto

In [260]:
import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


##### 3.1 Get location infomation from Toronto and create a map from the neighborhoods in Toronto

Use geopy module to get the geographic coordinates of Toronto

In [261]:
address = 'Toronto, Ontario'

geolocator = geopy.Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


Input the relavant account information for Foursquare to get the credentails

In [237]:
CLIENT_ID = 'AAGM420Z2D0G102XKRKQC2J5TWW3PNJNOIFNTTYX1Z1ZHBLS' # your Foursquare ID
CLIENT_SECRET = 'JQZHTIGGI14MFDOWR4UFTLNMFTLYTBHOTCMARIZPYZFFQPU5' # your Foursquare Secret
VERSION = '20200505' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AAGM420Z2D0G102XKRKQC2J5TWW3PNJNOIFNTTYX1Z1ZHBLS
CLIENT_SECRET:JQZHTIGGI14MFDOWR4UFTLNMFTLYTBHOTCMARIZPYZFFQPU5


Use the function built from lab to get information of venues nearby each neighborhood

In [315]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 5
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'NBH Latitude', 
                  'NBH Longitude', 
                  'Venue', 
                  'VN Latitude', 
                  'VN Longitude', 
                  'Category']
    
    return(nearby_venues)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Filter the neighborhoods in boroughs that only contain the word 'Toronto'

In [274]:
toronto_data = df4.set_index('Borough').filter(regex=".*Toronto",axis=0)
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District , Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosed

Create the map of above filtered neighborhoods with folium  module
(I think the map could not be displayed in Github, so you can find it here: https://drive.google.com/file/d/1TOJzVugFliDMY9AMOaBEtHOYPdb1ibk_/view )

In [275]:
map_to = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

Estimate the venues shape in Toronto and calculate the number of unique venues categories

In [276]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Category'].unique())))

(1204, 7)
There are 218 uniques categories.


##### 3.2 Explore the top 10 most common venues nearby each neighborhood

Data processing: one hot encoding

In [277]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the venues by neighborhoods and calculate the shape

In [278]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(39, 218)

Estimate the frequency of top 5 venues by neighborhoods

In [279]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2         Cheese Shop  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2       Nightclub  0.09
3  Breakfast Spot  0.09
4             Gym  0.04


----Business reply mail Processing Centre----
                venue  freq
0         Yoga Studio  0.06
1                 Spa  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.08
2  

Expand the number to top 10 venues of neighborhoods and create the dataframe

In [343]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{} th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Cheese Shop,Restaurant,Bakery,Café,Clothing Store,Japanese Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Stadium,Convenience Store,Pet Store,Climbing Gym,Burrito Place,Restaurant
2,Business reply mail Processing Centre,Yoga Studio,Burrito Place,Farmers Market,Spa,Auto Workshop,Recording Studio,Smoke Shop,Skate Park,Gym / Fitness Center,Pizza Place
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Harbor / Marina,Rental Car Location,Bar,Boutique,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Department Store,Spa,Falafel Restaurant


##### 3.3 Clustering the neighborhood with k-means algorithm

Set up parameters for k-means clustering

In [344]:
# set number of clusters
k = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0,
       0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Insert the k-means cluster labels into the dataframe

In [345]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Label', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Neighborhood,Latitude,Longitude,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,,,,,
Downtown Toronto,M5A,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Mexican Restaurant,Restaurant,Playground
Downtown Toronto,M7A,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Café,Hobby Shop,Bank,Bar,Discount Store,Yoga Studio,Mexican Restaurant
Downtown Toronto,M5B,"Garden District , Ryerson",43.657162,-79.378937,0,Café,Middle Eastern Restaurant,Restaurant,Clothing Store,Tea Room,Theater,Coffee Shop,Cosmetics Shop,Bookstore,Ramen Restaurant
Downtown Toronto,M5C,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Gastropub,Seafood Restaurant,Hotel,Cosmetics Shop,Creperie,American Restaurant,Optical Shop,Jazz Club
East Toronto,M4E,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Wine Shop,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


Create clustering map

(As already stated, you can find the map here:  https://drive.google.com/file/d/1EyxrY6wyez2wMeHQD_CDYtsaKbG72RJM/view )

In [346]:
# create map
latitude = 43.654260
longitude = -79.520999

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Retrieve data from each clustered neighborhood in details by sorted labels

##### Cluster 1

In [347]:
toronto_merged.loc[toronto_merged['Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Downtown Toronto,"Regent Park , Harbourfront",Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Mexican Restaurant,Restaurant,Playground
Downtown Toronto,"Queen's Park , Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Café,Hobby Shop,Bank,Bar,Discount Store,Yoga Studio,Mexican Restaurant
Downtown Toronto,"Garden District , Ryerson",Café,Middle Eastern Restaurant,Restaurant,Clothing Store,Tea Room,Theater,Coffee Shop,Cosmetics Shop,Bookstore,Ramen Restaurant
Downtown Toronto,St. James Town,Café,Coffee Shop,Gastropub,Seafood Restaurant,Hotel,Cosmetics Shop,Creperie,American Restaurant,Optical Shop,Jazz Club
East Toronto,The Beaches,Trail,Health Food Store,Pub,Wine Shop,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Cheese Shop,Restaurant,Bakery,Café,Clothing Store,Japanese Restaurant
Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Department Store,Spa,Falafel Restaurant
Downtown Toronto,Christie,Grocery Store,Café,Park,Baby Store,Candy Store,Restaurant,Diner,Italian Restaurant,Athletics & Sports,Coffee Shop
Downtown Toronto,"Richmond , Adelaide , King",Coffee Shop,Café,Concert Hall,Seafood Restaurant,Restaurant,American Restaurant,Pizza Place,Steakhouse,Opera House,Breakfast Spot


##### Cluster 2

In [348]:
toronto_merged.loc[toronto_merged['Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,Lawrence Park,Park,Construction & Landscaping,Swim School,Bus Line,Comfort Food Restaurant,Comic Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
Central Toronto,Forest Hill North & West,Bus Line,Sushi Restaurant,Park,Jewelry Store,Trail,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
Downtown Toronto,Rosedale,Park,Playground,Trail,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


##### Cluster 3

In [349]:
toronto_merged.loc[toronto_merged['Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,"Moore Park , Summerhill East",Park,Wine Shop,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


##### Cluster 4

In [350]:
toronto_merged.loc[toronto_merged['Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,Roselawn,Garden,Wine Shop,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


##### Some findings and explanations of the clustering process:

As we can see, the neighborhoods are grouped into 4 clusters:

Cluster 1 : the majority, their most common venues are mostly daily life facilities or gastronomies (e.g. Cafés, restaurants, shops...)  
Cluster 2 : contains 3 neighborhoods, where the most popular venues are parks and bus line  
Cluster 3 : contains only 1 neighborhood 'Moore Park , Summerhill East', where the most common venue is park (just the same as group 2)   
Cluster 4 : contains only 1 neighborhood 'Roselawn', where the most common venue is garden   

I personally think cluster 2 and 3 can be sorted into the same group, as they have very similar venues structures.  
Moreover, several neighborhoods could not be correctly categorized, for example neighborhood 'Davisville North' (in Borough Central Toronto) from group 1, whose most common venue is park, which should be grouped into cluster 2 or 3.